In [1]:
import pandas as pd

# Merging the same infobox
excel_path = pd.ExcelFile('D:\Kuliah\Skripsi\Ontologi-Sejarah-Indonesia\MichaelDawBalma\Code\infoboxes\infobox-orla-fix.xlsx')
sheet_name = pd.read_excel(excel_path, 'partai politik')
sheet_name2 = pd.read_excel(excel_path, 'partai politik Indonesia')
sheet_name3 = pd.read_excel(excel_path, 'Partai Politik1')
sheet_name4 = pd.read_excel(excel_path, 'political party')
sheet_name5 = pd.read_excel(excel_path, 'Political party1')

csv_file_path = 'partai-politik.csv'

merged_df = pd.concat([sheet_name, sheet_name2, sheet_name3, sheet_name4, sheet_name5])

merged_df.to_csv(csv_file_path, index=False)

In [224]:
import csv
partai_politik_clean = []

with open(csv_file_path, newline='', encoding='utf-8') as csvfile:
    reader = csv.DictReader(csvfile)

    for row in reader:
        partai_politik_clean.append(dict(row))

### Add more party data

In [17]:
sheets_dict = pd.read_excel(excel_path, sheet_name=None)

party_list = []
for sheet_name, df in sheets_dict.items():
    if 'party' in df.columns:
        party_column = df['party'].dropna()
        party_list.append(party_column.tolist())

# Write the list to a CSV file, then cleaning it manually
with open('additional_party_value.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['party'])
    for sublist in party_list:
        for item in sublist:
            writer.writerow([item])

In [46]:
import csv
additional_data_partai = []

with open('additional_party_value.csv', newline='', encoding='utf-8') as csvfile:
    reader = csv.DictReader(csvfile)

    for row in reader:
        additional_data_partai.append(dict(row))

In [220]:
import pywikibot, mwparserfromhell

additional_data_partai_infoboxes = {}
def get_infobox(page_title):
    """
    :param page_title: 
    :return: 
    
    Get infobox based on page title. 
    """
    # eliminates the separation using space between words in page_title
    global information
    
    if page_title == 'Persatuan Arab Indonesia':
        return 

    if len(page_title.split()) > 1 :
        page_title = page_title.replace(" ", "_")

    site = pywikibot.Site('id', 'wikipedia')
    page = pywikibot.Page(site, page_title)

    try :
        text = page.get()
        wikitext = mwparserfromhell.parse(text)
        infoboxes = wikitext.filter_templates(matches = 'Infobox')
        for infobox in infoboxes :
            information = {param.name.strip_code(keep_template_params = True).strip(): param.value.strip_code(keep_template_params = True).strip() for param in infobox.params}
            # save the infobox's name and its title as the first and second 'index' respectively 
            information = {'infobox_name' : str(infobox.name).replace('\n', ''), 'page_title' : page_title.replace("_", " "), **information}

            # Append to the list of values
            if page_title.replace("_", " ") in additional_data_partai_infoboxes:
                print("double", page_title)
                additional_data_partai_infoboxes[page_title.replace("_", " ")].append(information)
            # Make the values as list since the page can have multiple infoboxes
            else :
                additional_data_partai_infoboxes[page_title.replace("_", " ")] = [information]

    except pywikibot.page._basepage.IsRedirectPageError as e :
        print(f"!!! Title {page_title} has {e} Exception !!!")

In [221]:
for idx, value in enumerate(additional_data_partai):
    if value['page_title'] is not None:
        try:
            get_infobox(value['page_title'])
        except ValueError or TypeError:
            print(value['party'])

double Partai_Golongan_Karya
double Partai_Golongan_Karya
double Partai_Golongan_Karya
double Nahdlatul_Ulama
PNI Supeni
Partai Gerakan Indonesia Merdeka
Partai Komunis Belanda
Partai Persatuan Indonesia (Parpindo)
Gerakan Rakyat Baru
Pemuda Republik Indonesia


In [230]:
for i, val in enumerate(additional_data_partai_infoboxes):
    try :
        partai_politik_clean.append(additional_data_partai_infoboxes[val][0])
    except KeyError:
        partai_politik_clean.append(additional_data_partai_infoboxes[val])

### Add wikiurl, summary

In [231]:
import wikipedia

def retrieve_paragraph(article_title):
    wikipedia.set_lang("id")
    return wikipedia.summary(article_title, auto_suggest=False)

def get_url_wikipedia(article_title):
    wikipedia.set_lang("id")
    page = wikipedia.page(article_title, auto_suggest=False)
    return page.url

In [232]:
for index, data in enumerate(partai_politik_clean):
    # if the title has more than paragraph summary, get the first summary
    summary = retrieve_paragraph(data['page_title']).split("\n")
    data['summary'] = summary[0]

    wikiurl = get_url_wikipedia(data['page_title'])
    data['wikiurl'] = wikiurl

### Extract to CSV

In [233]:
all_keys = set()
for data in partai_politik_clean:
    all_keys.update(data.keys())

fieldnames = sorted(list(all_keys))

with open('partai-politik-cleaned.csv', 'w', newline='', encoding='utf-8') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    writer.writerows(partai_politik_clean)

### Multivalue

In [267]:
partai_politik_clean_all = []

partai_politik_multivalue_clean = []
partai_politik_non_multivalue_clean = []

with open('partai-politik-cleaned.csv', newline='', encoding='utf-8') as csvfile:
    reader = csv.DictReader(csvfile)

    for row in reader:
        partai_politik_clean_all.append(dict(row))

partai_politik_multivalue_clean_keys = ['chairman', 'founder', 'ideology', 'merged', 'position', 'predecessor', 'successor', 'think_tank', 'national', 'regional', 'continental', 'international', 'european', 'student_wing', 'vice_chairperson', 'youth_wing', 'women_wing', 'page_title']
partai_politik_non_multivalue_clean_keys = [key for key in partai_politik_clean_all[0] if key not in partai_politik_multivalue_clean_keys]
partai_politik_non_multivalue_clean_keys.append('page_title')

for item in partai_politik_clean_all:
    dict_A = {key: item[key] for key in partai_politik_multivalue_clean_keys}
    partai_politik_multivalue_clean.append(dict_A)

    dict_B = {key: item[key] for key in partai_politik_non_multivalue_clean_keys}
    partai_politik_non_multivalue_clean.append(dict_B)

In [269]:
fieldnames = partai_politik_non_multivalue_clean[0].keys()
with open('partai-politik-non-multivalue-cleaned.csv', 'w', newline='', encoding='utf-8') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    writer.writerows(partai_politik_non_multivalue_clean)

fieldnames = partai_politik_multivalue_clean[0].keys()
with open('partai-politik-multivalue-cleaned.csv', 'w', newline='', encoding='utf-8') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    writer.writerows(partai_politik_multivalue_clean)
